In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

In [2]:
DATA_DIR = "data/"

In [3]:
data = pd.read_excel(DATA_DIR+"Database_matrix_modified.xlsx",header=[0,1,2,3,4])

In [4]:
data.head()

Subject id                 Age             Gender     Marks in 10th   \
            Unnamed: 0_level_1 Unnamed: 1_level_1 Unnamed: 2_level_1   
            Unnamed: 0_level_2 Unnamed: 1_level_2 Unnamed: 2_level_2   
            Unnamed: 0_level_3 Unnamed: 1_level_3 Unnamed: 2_level_3   
            Unnamed: 0_level_4 Unnamed: 1_level_4 Unnamed: 2_level_4   
S1                        19.0                  M                NaN   
NaN                        NaN                NaN                NaN   
S2                        20.0                  M                9.8   
NaN                        NaN                NaN                NaN   
S3                        21.0                  M               10.0   

Subject id      Marks in 12th            JEE Rank               CGPA  \
            Unnamed: 3_level_1 Unnamed: 4_level_1 Unnamed: 5_level_1   
            Unnamed: 3_level_2 Unnamed: 4_level_2 Unnamed: 5_level_2   
            Unnamed: 3_level_3 Unnamed: 4_level_3 Unnamed: 5_level_3   
            Unnamed: 3_level_4 Unnamed: 4_level_4 Unnamed: 5_level_4   
S1                         NaN                NaN                  7   
NaN                        NaN                NaN                NaN   
S2                        87.8             1545.0               8.03   
NaN                        NaN                NaN                NaN   
S3                        93.4              436.0                8.7   

Subject id  Marks in PDS theory   Marks in PDS lab                Game  \
             Unnamed: 6_level_1 Unnamed: 7_level_1 Selective Attention   
             Unnamed: 6_level_2 Unnamed: 7_level_2       Count letters   
             Unnamed: 6_level_3 Unnamed: 7_level_3               score   
             Unnamed: 6_level_4 Unnamed: 7_level_4  Unnamed: 8_level_4   
S1                            B                  B                 0.0   
NaN                         NaN                NaN                 NaN   
S2                            A                  A                 0.0   
NaN                         NaN                NaN                 NaN   
S3                            A                  A                 5.0   

Subject id                             ...                        Code  \
                                       ...            Long Code Normal   
                                       ...                         LOC   
                       time(s)         ...         Unnamed: 37_level_3   
            Unnamed: 9_level_4         ...         Unnamed: 37_level_4   
S1                        28.7         ...                        39.0   
NaN                        NaN         ...                        43.0   
S2                        50.0         ...                        43.0   
NaN                        NaN         ...                         NaN   
S3                        47.8         ...                        51.0   

Subject id                                                                 \
                                                          Long Code Error   
            Cyclomatic Complexity                Time              Code #   
              Unnamed: 38_level_3 Unnamed: 39_level_3 Unnamed: 40_level_3   
              Unnamed: 38_level_4 Unnamed: 39_level_4 Unnamed: 40_level_4   
S1                            7.0                4.47                37.0   
NaN                           3.0                4.51                 NaN   
S2                            3.0                4.36                17.0   
NaN                           NaN                 NaN                34.0   
S3                            2.0                5.57                34.0   

Subject id                                                               \
                                                                          
            Number of questions               Score    Error Identified   
            Unnamed: 41_level_3 Unnamed: 42_level_3 Unnamed: 43_level_3   
     

In [5]:
data = np.array(data)

In [6]:
data[0]

array([19.0, 'M', nan, nan, nan, 7, 'B', 'B', 0.0, 28.7, 0.0, 37.4, 24.0,
       45.0, 20.0, 30.0, 10.0, 18.0, 0.0, 20.0, 0.0, 5.0, 0.0, 4.0, 0.0,
       3.0, 0.0, 3.0, 5.0, 4.0, 2.0, 21.0, 4.0, 1.23, 10.0, 4.0, 2.0,
       39.0, 7.0, 4.47, 37.0, 2.0, 2.0, 'y', 61.0, 2.0, 16.06],
      dtype=object)

In [7]:
game_features = data[:,8:28]
cols_list = [0,29,30,31,32,33,35,36,\
                 37,38,39,41,42,43,44,45,46]
code_features = data[:,cols_list]

In [8]:
code_features[0:2]

array([[19.0, 4.0, 2.0, 21.0, 4.0, 1.23, 4.0, 2.0, 39.0, 7.0, 4.47, 2.0,
        2.0, 'y', 61.0, 2.0, 16.06],
       [nan, 2.0, 2.0, 17.0, 2.0, 4.54, 4.0, 2.0, 43.0, 3.0, 4.51, nan,
        nan, nan, nan, nan, nan]], dtype=object)

In [9]:
def get_game_features(data):
    X_game = []
    for r in data:
        if not r[0] != r[0]:
            X_game.append(r)
    return np.array(X_game,dtype=np.float64)

In [10]:
def clean_code_features(data):
    for r in data:
        time_indices = [5, 10, 16]
        for j in time_indices:
            if r[j] != r[j]:
                continue
            if 'sec' in str(r[j]):
                r[j] = str(r[j])[:-3]
            else:
                mi = int(r[j])
                sec = int((float(r[j])-mi)*100)
                r[j] = mi*60+sec
    return data
def get_code_features(data):
    data = clean_code_features(data)
    X_code = []
    ques_num = 0
    for r in data:
        if not r[0] != r[0]:
            ques_num = 0
            X_code.append(np.zeros(5*6))
        if not r[1] != r[1]:
            X_code[len(X_code)-1][ques_num*6:ques_num*6+5] = r[1:6]
            ques_num += 1
        if not r[6] != r[6]:
            X_code[len(X_code)-1][ques_num*6:ques_num*6+5] = r[6:11]
            ques_num += 1
        if not r[11] != r[11]:
            X_code[len(X_code)-1][ques_num*6:ques_num*6+2] = r[11:13]
            X_code[len(X_code)-1][ques_num*6+2:ques_num*6+5] = r[14:17]
            if r[13]=='y':
                X_code[len(X_code)-1][ques_num*6+5] = 1
            ques_num += 1
    return np.array(X_code)

In [11]:
X_code = get_code_features(code_features)

In [12]:
X_game = get_game_features(game_features)
'''
for r in data:
    if not r[0] != r[0]:
        X_data.append(r)
X_data = np.array(X_data,dtype=np.float64)
'''

'\nfor r in data:\n    if not r[0] != r[0]:\n        X_data.append(r)\nX_data = np.array(X_data,dtype=np.float64)\n'

In [13]:
X_game = np.delete(X_game,[5,7],axis=1)
X_game_mean = np.mean(X_game,axis=0)
X_game_std = np.std(X_game,axis=0)
X_game = (X_game - X_game_mean)/X_game_std

In [14]:
X_game_std

array([ 2.13437475, 29.63262606,  1.67610496, 30.40746041,  9.92191738,
        9.3432487 ,  2.61321166,  8.88472684,  3.70708179,  7.97291712,
       10.42904384,  0.86463834,  5.2111246 ,  0.8640432 ,  6.93690455,
        1.09697802,  5.22165777,  0.77357772])

In [15]:
X_code = np.delete(X_code,[5,11],axis=1)
X_code_mean = np.mean(X_code,axis=0)
X_code_std = np.std(X_code,axis=0)
X_code = (X_code - X_code_mean)/X_code_std

In [16]:
X_code_std

array([1.31729031e+00, 1.25707872e+00, 6.36239811e+00, 7.87635938e-01,
       1.52833051e+02, 9.19421599e-01, 1.30591668e+00, 1.83190086e+01,
       5.18134779e+00, 1.73160912e+02, 9.42809042e-01, 1.04313682e+00,
       1.47990194e+01, 1.41336459e+00, 3.51847329e+02, 4.91351821e-01,
       1.09572683e+00, 1.15722221e+00, 2.89164695e+01, 2.17235720e+00,
       2.28820378e+02, 1.34816850e-01, 1.45673781e+00, 1.12887042e+00,
       2.22932199e+01, 1.20085132e+00, 2.18946483e+02, 4.56623259e-01])

In [17]:
def train_model(data,num_clusters=3,max_iter=1000):
    model = KMeans(num_clusters,max_iter=max_iter).fit(data)
    return model

In [18]:
NUM_CLUSTERS = 3

In [19]:
model_game = train_model(X_game,NUM_CLUSTERS,max_iter=1000)

In [20]:
model_game.labels_

array([0, 2, 1, 1, 1, 2, 0, 2, 2, 1, 2, 1, 0, 0, 0, 1, 0, 2, 1, 1, 1, 2,
       2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2,
       2, 1, 2, 1, 1, 2, 1, 0, 1, 1], dtype=int32)

In [21]:
lbls = [(i+1,x) for i,x in enumerate(model_game.labels_)]

In [34]:
clusters_map = {0:[], 1:[], 2:[]}

In [23]:
for lbl in lbls:
    clusters_map[lbl[1]].append("S"+str(lbl[0]))

In [24]:
print(clusters_map)

{0: ['S1', 'S7', 'S13', 'S14', 'S15', 'S17', 'S52'], 1: ['S3', 'S4', 'S5', 'S10', 'S12', 'S16', 'S19', 'S20', 'S21', 'S27', 'S29', 'S31', 'S32', 'S35', 'S36', 'S38', 'S41', 'S42', 'S46', 'S48', 'S49', 'S51', 'S53', 'S54'], 2: ['S2', 'S6', 'S8', 'S9', 'S11', 'S18', 'S22', 'S23', 'S24', 'S25', 'S26', 'S28', 'S30', 'S33', 'S34', 'S37', 'S39', 'S40', 'S43', 'S44', 'S45', 'S47', 'S50']}


In [35]:
model_code = train_model(X_code,NUM_CLUSTERS,max_iter=1000)

In [36]:
model_code.labels_

array([2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 0,
       2, 2, 2, 2, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 1, 2, 2, 1, 1, 1, 1, 1, 1], dtype=int32)

In [37]:
lbls2 = [(i+1,x) for i,x in enumerate(model_code.labels_)]
for lbl in lbls2:
    clusters_map[lbl[1]].append("S"+str(lbl[0]))
clusters_map

{0: ['S3',
  'S4',
  'S5',
  'S6',
  'S7',
  'S9',
  'S10',
  'S12',
  'S13',
  'S14',
  'S17',
  'S22',
  'S28'],
 1: ['S29',
  'S30',
  'S31',
  'S32',
  'S33',
  'S34',
  'S35',
  'S36',
  'S37',
  'S38',
  'S39',
  'S40',
  'S41',
  'S42',
  'S46',
  'S49',
  'S50',
  'S51',
  'S52',
  'S53',
  'S54'],
 2: ['S1',
  'S2',
  'S8',
  'S11',
  'S15',
  'S16',
  'S18',
  'S19',
  'S20',
  'S21',
  'S23',
  'S24',
  'S25',
  'S26',
  'S27',
  'S43',
  'S44',
  'S45',
  'S47',
  'S48']}

In [28]:
model_code.cluster_centers_*X_code_std + X_code_mean

array([[2.33333333e+00, 1.66666667e+00, 1.76666667e+01, 2.33333333e+00,
        4.35166667e+02, 3.83333333e+00, 1.33333333e+00, 5.90000000e+01,
        4.66666667e+00, 3.63500000e+02, 2.16666667e+00, 1.16666667e+00,
        5.00000000e+01, 4.00000000e+00, 4.59666667e+02, 3.33333333e-01,
        3.00000000e+00, 2.33333333e+00, 9.45000000e+01, 3.33333333e+00,
        6.68833333e+02, 0.00000000e+00, 4.16666667e+00, 3.16666667e+00,
        6.86666667e+01, 3.50000000e+00, 3.98333333e+02, 3.33333333e-01],
       [3.05882353e+00, 2.41176471e+00, 2.52352941e+01, 3.05882353e+00,
        1.54588235e+02, 5.17647059e+00, 3.64705882e+00, 6.16470588e+01,
        4.23529412e+00, 3.61823529e+02, 3.52941176e+00, 2.52941176e+00,
        6.31176471e+01, 3.58823529e+00, 4.57176471e+02, 5.29411765e-01,
        4.05882353e+00, 2.88235294e+00, 6.67647059e+01, 3.29411765e+00,
        3.40058824e+02, 0.00000000e+00, 3.47058824e+00, 2.41176471e+00,
        6.55294118e+01, 3.11764706e+00, 3.42411765e+02, 2.94117

In [29]:
model_game_code = train_model(np.concatenate((X_code,X_game),axis=1),NUM_CLUSTERS,max_iter=1000)

In [30]:
lbls = [(i+1,x) for i,x in enumerate(model_game_code.labels_)]

In [31]:
model_game_code.labels_

array([2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 1, 0, 2, 2, 2, 2, 0, 2,
       2, 2, 1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)